In [276]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import os

# 1. Data Loading and Pre-processing

In [277]:
data = pd.read_csv("drug200.csv")
data.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY


In [278]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

## Encoding

In [279]:
# sex, BP, Cholesterol, Drug columns encoding

# sex column
le = LabelEncoder()
X[:, 1] = le.fit_transform(X[:, 1])

# BP, Cholestero columns
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [2, 3])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

# Drug column
encoder = OneHotEncoder()
encoded_Y = encoder.fit(y.reshape(-1,1))
y = encoded_Y.transform(y.reshape(-1,1)).toarray()

## Split

In [280]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Scaling

In [281]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# 2. Artificial Neural Netwrok (ANN)

In [282]:
# hidden layers (2)

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=15, activation='relu'))
ann.add(tf.keras.layers.Dense(units=15, activation='relu'))

In [283]:
# output layer

number_of_possible_outcomes =  len(set(data["Drug"]))
ann.add(tf.keras.layers.Dense(units=number_of_possible_outcomes, activation='softmax'))

In [284]:
# compile

ann.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [285]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
5/5 [==============================] - 0s 1ms/step - loss: 1.5062 - accuracy: 0.2188
Epoch 2/100
5/5 [==============================] - 0s 1ms/step - loss: 1.4709 - accuracy: 0.2625
Epoch 3/100
5/5 [==============================] - 0s 1ms/step - loss: 1.4391 - accuracy: 0.3313
Epoch 4/100
5/5 [==============================] - 0s 1ms/step - loss: 1.4068 - accuracy: 0.3688
Epoch 5/100
5/5 [==============================] - 0s 1ms/step - loss: 1.3781 - accuracy: 0.3750
Epoch 6/100
5/5 [==============================] - 0s 1ms/step - loss: 1.3494 - accuracy: 0.3938
Epoch 7/100
5/5 [==============================] - 0s 1ms/step - loss: 1.3229 - accuracy: 0.4313
Epoch 8/100
5/5 [==============================] - 0s 1ms/step - loss: 1.2966 - accuracy: 0.4375
Epoch 9/100
5/5 [==============================] - 0s 1ms/step - loss: 1.2713 - accuracy: 0.4375
Epoch 10/100
5/5 [==============================] - 0s 1ms/step - loss: 1.2462 - accuracy: 0.4500
Epoch 11/100
5/5 [===========

5/5 [==============================] - 0s 1ms/step - loss: 0.2015 - accuracy: 0.9625
Epoch 85/100
5/5 [==============================] - 0s 1ms/step - loss: 0.1970 - accuracy: 0.9563
Epoch 86/100
5/5 [==============================] - 0s 1ms/step - loss: 0.1924 - accuracy: 0.9563
Epoch 87/100
5/5 [==============================] - 0s 1ms/step - loss: 0.1883 - accuracy: 0.9563
Epoch 88/100
5/5 [==============================] - 0s 1ms/step - loss: 0.1842 - accuracy: 0.9563
Epoch 89/100
5/5 [==============================] - 0s 1ms/step - loss: 0.1807 - accuracy: 0.9563
Epoch 90/100
5/5 [==============================] - 0s 1ms/step - loss: 0.1764 - accuracy: 0.9563
Epoch 91/100
5/5 [==============================] - 0s 1ms/step - loss: 0.1723 - accuracy: 0.9688
Epoch 92/100
5/5 [==============================] - 0s 1ms/step - loss: 0.1687 - accuracy: 0.9688
Epoch 93/100
5/5 [==============================] - 0s 1ms/step - loss: 0.1649 - accuracy: 0.9625
Epoch 94/100
5/5 [===============

# 3. Prediction for test data

In [286]:
y_pred = ann.predict(X_test)

2/2 [==============================] - 0s 2ms/step


In [287]:
# conver prediction outcome to the actual values

y_pred_outcome = []
for output in y_pred:
    output_list = [0] * number_of_possible_outcomes
    output_list[np.where(output == max(output))[0][0]] = 1
    y_pred_outcome.append(output_list)

In [288]:
accuracy_score(y_test, y_pred_outcome)

1.0